In [1]:
import requests
import pandas as pd
import geopandas as gpd
from shapely.geometry import Point

# Fetch the GeoJSON data
url = "https://earthquake.usgs.gov/fdsnws/event/1/query.geojson?starttime=1975-01-01%2000:00:00&endtime=2025-01-10%2023:59:59&maxlatitude=50&minlatitude=24.6&maxlongitude=-65&minlongitude=-125&minmagnitude=4.5&orderby=time"
response = requests.get(url)
data = response.json()

In [2]:
# Parse the GeoJSON into a pandas DataFrame
features = data['features']

# Extract relevant data
earthquakes = [
    {
        "id": feature["id"],
        "magnitude": feature["properties"]["mag"],
        "place": feature["properties"]["place"],
        "time": pd.to_datetime(feature["properties"]["time"], unit='ms'),
        "updated": pd.to_datetime(feature["properties"]["updated"], unit='ms'),
        "latitude": feature["geometry"]["coordinates"][1],
        "longitude": feature["geometry"]["coordinates"][0],
        "depth": feature["geometry"]["coordinates"][2],
    }
    for feature in features
]

In [3]:
# Convert to DataFrame
df = pd.DataFrame(earthquakes)

# Display the first few rows
df

,id,magnitude,place,time,updated,latitude,longitude,depth
0,nc75111126,4.65,"4 km NW of Cobb, CA",2025-01-02 02:34:04.660,2025-01-14 02:44:10.574,38.845500,-122.756333,1.420
1,nn00888580,5.70,"2024 Parker Butte, Nevada Earthquake",2024-12-09 23:08:31.577,2025-01-13 23:18:44.633,39.167500,-119.023800,9.300
2,nc75001903,4.67,"53 km WNW of Petrolia, CA",2024-12-05 18:46:29.660,2024-12-21 15:56:00.930,40.349000,-124.915000,12.510
3,ci40964384,4.54,"10 km SE of Stovepipe Wells, CA",2024-10-25 15:04:23.620,2025-01-04 23:08:10.040,36.550833,-117.056000,4.580
4,ci40964128,4.72,"5 km E of Stovepipe Wells, CA",2024-10-25 08:05:08.290,2025-01-10 08:15:13.389,36.601667,-117.091167,3.980
...,...,...,...,...,...,...,...,...
1904,usp0000a2j,5.70,"65 km SSW of Rachel, Nevada",1975-02-28 15:15:00.000,2016-01-29 22:01:18.960,37.106000,-116.056000,0.000
1905,usp00009sj,4.60,"12 km E of Evergreen, Montana",1975-02-04 01:32:52.100,2020-08-26 00:08:48.246,48.213000,-114.110000,8.000
1906,ci3037458,4.84,"4km SE of Brawley, CA",1975-01-23 17:02:30.020,2018-06-04 20:43:45.000,32.952000,-115.494000,13.560
1907,ci3037348,4.84,"32km ESE of San Clemente Is. (SE tip), CA",1975-01-12 21:22:13.480,2018-06-04 20:43:45.000,32.693000,-118.040000,6.000


In [4]:
# Save the DataFrame to a JSON file
output_path = "outputs/samples.json"
df.to_json(output_path, orient="records", indent=2)

In [7]:
# Create GeoDataFrame
geometry = [Point(xy) for xy in zip(df["longitude"], df["latitude"])]
gdf = gpd.GeoDataFrame(df, geometry=geometry)


In [8]:
# Filter earthquakes within a shapefile (e.g., US boundary)
continental_us = gpd.read_file("path_to_us_shapefile.shp")
us_quakes = gdf[gdf.within(continental_us.unary_union)]

# Plot earthquakes
us_quakes.plot()

ImportError: The 'read_file' function requires the 'pyogrio' or 'fiona' package, but neither is installed or imports correctly.
Importing fiona resulted in: dlopen(/opt/anaconda3/envs/dev/lib/python3.10/site-packages/fiona/_env.cpython-310-darwin.so, 0x0002): Library not loaded: @rpath/libpoppler.126.dylib
  Referenced from: <32AB0042-17C3-304F-B97F-B43936E694E3> /opt/anaconda3/envs/dev/lib/libgdal.32.3.6.2.dylib
  Reason: tried: '/opt/anaconda3/envs/dev/lib/libpoppler.126.dylib' (no such file), '/opt/anaconda3/envs/dev/lib/python3.10/site-packages/fiona/../../../libpoppler.126.dylib' (no such file), '/opt/anaconda3/envs/dev/lib/python3.10/site-packages/fiona/../../../libpoppler.126.dylib' (no such file), '/opt/anaconda3/envs/dev/bin/../lib/libpoppler.126.dylib' (no such file), '/opt/anaconda3/envs/dev/bin/../lib/libpoppler.126.dylib' (no such file), '/usr/local/lib/libpoppler.126.dylib' (no such file), '/usr/lib/libpoppler.126.dylib' (no such file, not in dyld cache)
Importing pyogrio